
1. 환경 설정 및 라이브러리 설치



In [ ]:
!pip install transformers datasets accelerate evaluate

import os
import torch
import numpy as np
from transformers import BertTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from evaluate import load as load_metric
from google.colab import drive

1.1 GPU 설정

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

1.2 구글 드라이브 마운트

In [ ]:
drive.mount('/content/drive')

2. 하이브리드 토크나이저 로드

In [ ]:
VOCAB_FILE = "/content/drive/MyDrive/bert/hybrid_bert_vocab.txt"

if not os.path.exists(VOCAB_FILE):
    raise FileNotFoundError(f" 오류: {VOCAB_FILE} 경로에 보캡 파일이 없습니다. 파일을 업로드하거나 경로를 수정하세요.")

# .txt 파일로 BertTokenizer 생성
tokenizer = BertTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)
print(f" 하이브리드 Vocab 로드 완료! 크기: {len(tokenizer)} (15,477개 예상)")


3. 데이터셋 로드 및 전처리


In [ ]:
datasets = load_dataset("dair-ai/emotion")

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# 토큰화 수행
tokenized_datasets = datasets.map(preprocess_function, batched=True)

# 불필요한 컬럼 제거 및 PyTorch 포맷 설정
columns_to_keep = ['input_ids', 'attention_mask', 'label']
columns_to_remove = [col for col in tokenized_datasets['train'].column_names if col not in columns_to_keep]
tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)
tokenized_datasets.set_format("torch")

print(" 데이터 전처리 완료.")

4. 모델 로드 및 임베딩 사이즈 조정

In [ ]:
MODEL_NAME = "bert-base-uncased"
num_labels = 6

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

print(f" 임베딩 사이즈 조정: {model.config.vocab_size} -> {len(tokenizer)}")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

5. 학습 설정 및 시작 (Trainer)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = load_metric("accuracy")
    return metric.compute(predictions=predictions, references=labels)

# 학습 파라미터 (표준 BERT에 최적화됨)
training_args = TrainingArguments(
    output_dir="./results_hybrid_finetuning",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    fp16=True,                      # GPU 가속 및 메모리 절약
    gradient_accumulation_steps=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()
